In [527]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import sqlite3

In [1115]:
pwd

'/Users/onlyone/Documents/GitHub/Correlates of War Project/War Data'

In [1116]:
ls

Extra-StateWarData_v4.0.csv      Non-StateWarData_v4.0.csv
Inter-StateWarData_v4.0.csv      correlates_of_war_project.ipynb
Intra-StateWarData_v4.1.csv


# Obtaning Data

In [1117]:
inter_state=pd.read_csv('Inter-StateWarData_v4.0.csv')

# Understanding the data

In [1118]:
# I will try to understand what I have and take some "to do" notes for "data wrangling" 

In [1119]:
inter_state.head()

,WarNum,WarName,WarType,ccode,StateName,Side,StartMonth1,StartDay1,StartYear1,EndMonth1,...,EndMonth2,EndDay2,EndYear2,TransFrom,WhereFought,Initiator,Outcome,TransTo,BatDeath,Version
0,1,Franco-Spanish War,1,230,Spain,2,4,7,1823,11,...,-8,-8,-8,503,2,2,2,-8,600,4
1,1,Franco-Spanish War,1,220,France,1,4,7,1823,11,...,-8,-8,-8,503,2,1,1,-8,400,4
2,4,First Russo-Turkish,1,640,Ottoman Empire,2,4,26,1828,9,...,-8,-8,-8,506,11,2,2,-8,80000,4
3,4,First Russo-Turkish,1,365,Russia,1,4,26,1828,9,...,-8,-8,-8,506,11,1,1,-8,50000,4
4,7,Mexican-American,1,70,Mexico,2,4,25,1846,9,...,-8,-8,-8,-8,1,2,2,-8,6000,4


In [1120]:
# Findings; 
#1 - meanings of some colums, like "ccode", "TransForm", "TransTo" are not clear
#2 - meanings of some values, like "-8" (might be instead of "NaN") and the values of "WhereFought" are not clear.
#2 - there are 12 colums for datetime, needs to be reduced
#3 - some colums need to be removed, like "Version"

In [1121]:
inter_state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 25 columns):
WarNum         337 non-null int64
WarName        337 non-null object
WarType        337 non-null int64
ccode          337 non-null int64
StateName      337 non-null object
Side           337 non-null int64
StartMonth1    337 non-null int64
StartDay1      337 non-null int64
StartYear1     337 non-null int64
EndMonth1      337 non-null int64
EndDay1        337 non-null int64
EndYear1       337 non-null int64
StartMonth2    337 non-null int64
StartDay2      337 non-null int64
StartYear2     337 non-null int64
EndMonth2      337 non-null int64
EndDay2        337 non-null int64
EndYear2       337 non-null int64
TransFrom      337 non-null object
WhereFought    337 non-null int64
Initiator      337 non-null int64
Outcome        337 non-null int64
TransTo        337 non-null int64
BatDeath       337 non-null int64
Version        337 non-null int64
dtypes: int64(22), object(3)
memory usage:

In [1122]:
# I have 12 columns about datetime but not one is in type of datetime,
# we need to create at least two datetime columns with the type of datetime for start and end time informations
# it looks there is no "null" but we need to check "-8" issue. 
# There should be some NaN values, at least for the Wars which has no second start time, so second end time. 

In [1123]:
inter_state.describe()

,WarNum,WarType,ccode,Side,StartMonth1,StartDay1,StartYear1,EndMonth1,EndDay1,EndYear1,...,StartYear2,EndMonth2,EndDay2,EndYear2,WhereFought,Initiator,Outcome,TransTo,BatDeath,Version
count,337.000000,337.0,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,...,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,3.370000e+02,337.0
mean,126.919881,1.0,419.908012,1.418398,6.249258,15.353116,1930.572700,6.623145,15.035608,1931.827893,...,100.281899,-7.183976,-6.433234,100.290801,4.925816,1.676558,2.091988,47.112760,9.519555e+04,4.0
std,61.237069,0.0,246.287700,0.494030,3.278294,8.597650,46.607801,3.100086,8.368658,46.703536,...,443.782753,3.412422,6.719895,443.819909,3.475743,0.468485,1.527670,189.345361,5.011997e+05,0.0
min,1.000000,1.0,2.000000,1.000000,1.000000,1.000000,1823.000000,1.000000,1.000000,1823.000000,...,-8.000000,-8.000000,-8.000000,-8.000000,1.000000,1.000000,1.000000,-8.000000,-9.000000e+00,4.0
25%,82.000000,1.0,220.000000,1.000000,3.000000,7.000000,1900.000000,4.000000,10.000000,1900.000000,...,-8.000000,-8.000000,-8.000000,-8.000000,2.000000,1.000000,1.000000,-8.000000,4.000000e+02,4.0
50%,139.000000,1.0,355.000000,1.000000,6.000000,16.000000,1939.000000,7.000000,14.000000,1941.000000,...,-8.000000,-8.000000,-8.000000,-8.000000,6.000000,2.000000,2.000000,-8.000000,2.000000e+03,4.0
75%,172.000000,1.0,652.000000,2.000000,9.000000,23.000000,1969.000000,9.000000,23.000000,1973.000000,...,-8.000000,-8.000000,-8.000000,-8.000000,7.000000,2.000000,2.000000,-8.000000,1.000000e+04,4.0
max,227.000000,1.0,920.000000,2.000000,12.000000,31.000000,2003.000000,12.000000,31.000000,2003.000000,...,1974.000000,10.000000,31.000000,1974.000000,19.000000,2.000000,8.000000,877.000000,7.500000e+06,4.0


In [1124]:
# I noticed only one column which returns with meaningful outputs, "BatDeats". 

In [1125]:
# 2ND FILE = INTRA_STATE WARS

In [1126]:
intra_state=pd.read_csv('Intra-StateWarData_v4.1.csv', encoding='latin-1')

In [1127]:
intra_state.head()

,WarNum,WarName,WarType,CcodeA,SideA,CcodeB,SideB,Intnl,StartMonth1,StartDay1,...,EndDay2,EndYear2,TransFrom,WhereFought,Initiator,Outcome,TransTo,SideADeaths,SideBDeaths,Version
0,500,First Caucasus,5,365.0,Russia,-8.0,"Georgians, Dhagestania, Chechens",0,6,10,...,-8,-8,-8.0,2,Chechens,1,-8,5000,6000,4.1
1,501,Sidon-Damascus,6,-8.0,Sidon,-8.0,Damascus & Aleppo,0,6,-9,...,-8,-8,-8.0,6,Sidon,2,-8,-9,-9,4.1
2,502,First Two Sicilies,4,300.0,Austria,-8.0,-8,1,3,-9,...,-8,-8,-8.0,2,Liberals,1,-8,-9,-8,4.1
3,502,First Two Sicilies,4,329.0,Two Sicilies,-8.0,Liberals,1,7,2,...,-8,-8,-8.0,2,Liberals,1,-8,-9,-9,4.1
4,503,Spanish Royalists,4,230.0,Spain,-8.0,Royalists,0,12,1,...,-8,-8,-8.0,2,Royalists,4,1,-9,-9,4.1


In [1128]:
# Number columns and some informations are different, 
# And also stucture is different, in this file both side of each single war are in the same row. 
# so if we want to bring them together, we need to put them in the same sturcture

In [1129]:
intra_state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 28 columns):
WarNum         442 non-null int64
WarName        442 non-null object
WarType        442 non-null int64
CcodeA         439 non-null float64
SideA          442 non-null object
CcodeB         441 non-null float64
SideB          442 non-null object
Intnl          442 non-null int64
StartMonth1    442 non-null int64
StartDay1      442 non-null int64
StartYear1     442 non-null int64
EndMonth1      442 non-null int64
EndDay1        442 non-null int64
EndYear1       440 non-null float64
StartMonth2    442 non-null int64
StartDay2      442 non-null int64
StartYear2     442 non-null int64
EndMonth2      442 non-null int64
EndDay2        442 non-null int64
EndYear2       442 non-null int64
TransFrom      441 non-null float64
WhereFought    442 non-null int64
Initiator      427 non-null object
Outcome        442 non-null int64
TransTo        442 non-null int64
SideADeaths    442 non-null objec

In [1130]:
# similar situation with the previous file except some small amount of missings
# Additionally and interestingly, dtype of EndYear1 is "float64" and we need to change.
# Because we experienced before that ".describe()" method result is not so meaningful I will not run it for this file

# Scrubing/Cleaning

In [1131]:
### 1 ### Dropping some columns:
        #I will drop the columns which will not be meaningful or useful in this work

In [1132]:
intra_state.drop(['CcodeA', 'CcodeB', 'Intnl', 'StartMonth2', 'StartDay2', 'StartYear2', 'EndMonth2', 'EndDay2', 'EndYear2', 'TransFrom', 'TransTo', 'Version'], axis=1, inplace=True)

In [1133]:
intra_state.head(2)

,WarNum,WarName,WarType,SideA,SideB,StartMonth1,StartDay1,StartYear1,EndMonth1,EndDay1,EndYear1,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths
0,500,First Caucasus,5,Russia,"Georgians, Dhagestania, Chechens",6,10,1818,-9,-9,1822.0,2,Chechens,1,5000,6000
1,501,Sidon-Damascus,6,Sidon,Damascus & Aleppo,6,-9,1820,7,21,1821.0,6,Sidon,2,-9,-9


In [1134]:
### 2 ### Create two new "date columns" (start and end dates) from 6 different columns

    # date informations are scattered to different columns as "day", "month" and "year"
    # and also they have some missing values and data type problems need to be handled
    
    # 2.a # assigning "values" to missing values of date related columns
    
        # If there is no acceptable StartDay or StartMonth value I will pass "1" as first day or first month.
        # in order to deal with the error of "day is out of range for month" 
        # I will assing "28" for the days greater than 28 
        # for this purpose I wrote function

In [1135]:
def to_one(x, inplace=True):
    if x<0:
        return 1
    elif x>28:
        return 28
    else:
        return x

In [ ]:
    # 2.b # creating "day", "month" and "year" lists
    
        # actually, I tried to extract them directly to a datetime format (for hours) but it didnt work or I couldnt
        # in order to get them together i created new nparrays with filling missing values
        # the second issue was an error message about "day range", 
        # I limited the value of the days to handle this problem

In [1136]:
day=intra_state.StartDay1.apply(to_one).values

In [1137]:
month=intra_state.StartMonth1.apply(to_one).values

In [1138]:
year=intra_state.StartYear1.values

In [1139]:
df=pd.DataFrame({'year': year, 'month': month, 'day': day})

In [ ]:
    # 2.c # Creating a new "date column" for start date 

In [1140]:
intra_state['start_date']=pd.to_datetime(df)

In [1141]:
intra_state.head()

,WarNum,WarName,WarType,SideA,SideB,StartMonth1,StartDay1,StartYear1,EndMonth1,EndDay1,EndYear1,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths,start_date
0,500,First Caucasus,5,Russia,"Georgians, Dhagestania, Chechens",6,10,1818,-9,-9,1822.0,2,Chechens,1,5000,6000,1818-06-10
1,501,Sidon-Damascus,6,Sidon,Damascus & Aleppo,6,-9,1820,7,21,1821.0,6,Sidon,2,-9,-9,1820-06-01
2,502,First Two Sicilies,4,Austria,-8,3,-9,1821,3,23,1821.0,2,Liberals,1,-9,-8,1821-03-01
3,502,First Two Sicilies,4,Two Sicilies,Liberals,7,2,1820,3,23,1821.0,2,Liberals,1,-9,-9,1820-07-02
4,503,Spanish Royalists,4,Spain,Royalists,12,1,1821,4,6,1823.0,2,Royalists,4,-9,-9,1821-12-01


In [ ]:
    # 2.d # Creating another "date" column for end of war

In [1142]:
day1=intra_state.EndDay1.apply(to_one).values

In [1143]:
month1=intra_state.EndMonth1.apply(to_one).values

In [1144]:
        # since the "EndYear1" dtype is 'float64', we need to convert to integer
        # second issue with the "EndYear1", we have some negative values, they are 'NaN' values and need to be filled.

In [1145]:
year1=intra_state.EndYear1.values.astype(int)

In [1146]:
df1=pd.DataFrame({'year': year1, 'month': month1, 'day': day1})

In [1147]:
df1.year[df1.year<0] = np.nan

In [1148]:
df1.year.fillna(method='bfill', inplace=True)

In [1149]:
df1.year=df1.year.astype(int)

In [1150]:
intra_state['end_date']=pd.to_datetime(df1)

In [1151]:
intra_state.head()

,WarNum,WarName,WarType,SideA,SideB,StartMonth1,StartDay1,StartYear1,EndMonth1,EndDay1,EndYear1,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths,start_date,end_date
0,500,First Caucasus,5,Russia,"Georgians, Dhagestania, Chechens",6,10,1818,-9,-9,1822.0,2,Chechens,1,5000,6000,1818-06-10,1822-01-01
1,501,Sidon-Damascus,6,Sidon,Damascus & Aleppo,6,-9,1820,7,21,1821.0,6,Sidon,2,-9,-9,1820-06-01,1821-07-21
2,502,First Two Sicilies,4,Austria,-8,3,-9,1821,3,23,1821.0,2,Liberals,1,-9,-8,1821-03-01,1821-03-23
3,502,First Two Sicilies,4,Two Sicilies,Liberals,7,2,1820,3,23,1821.0,2,Liberals,1,-9,-9,1820-07-02,1821-03-23
4,503,Spanish Royalists,4,Spain,Royalists,12,1,1821,4,6,1823.0,2,Royalists,4,-9,-9,1821-12-01,1823-04-06


In [ ]:
### 3 ### Dropping the old columns

    # since we extracted the information we need and we have new date columns, we dont need the old ones

In [1152]:
intra_state.drop(['StartMonth1', 'StartDay1', 'StartYear1', 'EndMonth1', 'EndDay1', 'EndYear1'], axis=1, inplace=True)

In [1153]:
intra_state.head()

,WarNum,WarName,WarType,SideA,SideB,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths,start_date,end_date
0,500,First Caucasus,5,Russia,"Georgians, Dhagestania, Chechens",2,Chechens,1,5000,6000,1818-06-10,1822-01-01
1,501,Sidon-Damascus,6,Sidon,Damascus & Aleppo,6,Sidon,2,-9,-9,1820-06-01,1821-07-21
2,502,First Two Sicilies,4,Austria,-8,2,Liberals,1,-9,-8,1821-03-01,1821-03-23
3,502,First Two Sicilies,4,Two Sicilies,Liberals,2,Liberals,1,-9,-9,1820-07-02,1821-03-23
4,503,Spanish Royalists,4,Spain,Royalists,2,Royalists,4,-9,-9,1821-12-01,1823-04-06


In [1154]:
# we reduced our 25 columns to 11 columns with dropping and aggregating. 
# it may looks simple for many of you but it took my hours, especially finding the correct way to form a date columns
# now we have a different challange: seperating the SideA and SideB

In [ ]:
### 4 ### Melting the "sides of wars"

    # why we do this: the first thing, to get similar formats with other "Corolates of War" data sets
    # this is important to use this data sets togetherif we want to use this data set with other 
    # second thing, it will be useful during the analysis

In [1156]:
    # 4.a # Filling missing values
        # Before doing to seperate the SideA and SideB we need to assign NaN's to where needed

In [1157]:
intra_state.SideA[intra_state.SideA.str.contains('8')]=np.nan

/Users/onlyone/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1158]:
intra_state.SideB[intra_state.SideB.str.contains('8')]=np.nan

/Users/onlyone/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1159]:
intra_state.SideADeaths=intra_state.SideADeaths.iloc[:].str.replace(',', '').astype(int)

In [1160]:
intra_state.SideBDeaths=intra_state.SideBDeaths.iloc[:].str.replace(',', '').astype(int)

In [1161]:
intra_state.SideADeaths[intra_state.SideADeaths<0]=np.nan

/Users/onlyone/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1162]:
intra_state.SideBDeaths[intra_state.SideBDeaths<0]=np.nan

/Users/onlyone/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1169]:
intra_state.head(1)

,WarNum,WarName,WarType,SideA,SideB,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths,start_date,end_date
0,500,First Caucasus,5,Russia,"Georgians, Dhagestania, Chechens",2,Chechens,1,5000.0,6000.0,1818-06-10,1822-01-01


# Reshaping

In [1170]:
intra = pd.melt(intra_state, id_vars=['WarNum', 'WarName', 'WarType', 'WhereFought', 'Initiator', 'Outcome', 'SideADeaths', 'SideBDeaths', 
                              'start_date', 'end_date'], var_name='state', value_name='value')

In [1171]:
intra=intra.sort_values('WarNum')

In [1172]:
intra.head()

,WarNum,WarName,WarType,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths,start_date,end_date,state,value
0,500,First Caucasus,5,2,Chechens,1,5000.0,6000.0,1818-06-10,1822-01-01,SideA,Russia
442,500,First Caucasus,5,2,Chechens,1,5000.0,6000.0,1818-06-10,1822-01-01,SideB,"Georgians, Dhagestania, Chechens"
1,501,Sidon-Damascus,6,6,Sidon,2,NaN,NaN,1820-06-01,1821-07-21,SideA,Sidon
443,501,Sidon-Damascus,6,6,Sidon,2,NaN,NaN,1820-06-01,1821-07-21,SideB,Damascus & Aleppo
445,502,First Two Sicilies,4,2,Liberals,1,NaN,NaN,1820-07-02,1821-03-23,SideB,Liberals


In [1173]:
intra.rename(columns={'state': 'side', 'value': 'state'}, inplace=True)

In [1174]:
intra.head()

,WarNum,WarName,WarType,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths,start_date,end_date,side,state
0,500,First Caucasus,5,2,Chechens,1,5000.0,6000.0,1818-06-10,1822-01-01,SideA,Russia
442,500,First Caucasus,5,2,Chechens,1,5000.0,6000.0,1818-06-10,1822-01-01,SideB,"Georgians, Dhagestania, Chechens"
1,501,Sidon-Damascus,6,6,Sidon,2,NaN,NaN,1820-06-01,1821-07-21,SideA,Sidon
443,501,Sidon-Damascus,6,6,Sidon,2,NaN,NaN,1820-06-01,1821-07-21,SideB,Damascus & Aleppo
445,502,First Two Sicilies,4,2,Liberals,1,NaN,NaN,1820-07-02,1821-03-23,SideB,Liberals


In [1197]:
intra.reset_index().head()

,index,WarNum,WarName,WarType,WhereFought,Initiator,Outcome,SideADeaths,SideBDeaths,start_date,end_date,side,state,deaths
0,0,500,First Caucasus,5,2,Chechens,1,5000.0,6000.0,1818-06-10,1822-01-01,SideA,Russia,NaN
1,442,500,First Caucasus,5,2,Chechens,1,5000.0,6000.0,1818-06-10,1822-01-01,SideB,"Georgians, Dhagestania, Chechens",NaN
2,1,501,Sidon-Damascus,6,6,Sidon,2,NaN,NaN,1820-06-01,1821-07-21,SideA,Sidon,NaN
3,443,501,Sidon-Damascus,6,6,Sidon,2,NaN,NaN,1820-06-01,1821-07-21,SideB,Damascus & Aleppo,NaN
4,445,502,First Two Sicilies,4,2,Liberals,1,NaN,NaN,1820-07-02,1821-03-23,SideB,Liberals,NaN


In [1198]:
intra.deaths = (lambda x, y: x if intra.side=='SideA' else intra.side=='SideB', intra.SideADeaths, intra.SideBDeaths)   

In [1199]:
intra.deaths

(<function __main__.<lambda>(x, y)>, 0       5000.0
 442     5000.0
 1          NaN
 443        NaN
 445        NaN
 444        NaN
 2          NaN
 3          NaN
 4          NaN
 446        NaN
 6          NaN
 448        NaN
 5          NaN
 447        NaN
 449     6000.0
 450        NaN
 451        NaN
 452        NaN
 9          NaN
 7       6000.0
 8          NaN
 10         NaN
 11         NaN
 453        NaN
 454      300.0
 12       300.0
 13         NaN
 14     12000.0
 455        NaN
 456    12000.0
         ...   
 427        NaN
 869        NaN
 870      150.0
 428      150.0
 871     1260.0
 429     1260.0
 873     1200.0
 872        NaN
 431     1200.0
 430        NaN
 432        NaN
 874        NaN
 875      900.0
 433      900.0
 876      723.0
 434      723.0
 877      100.0
 435      100.0
 878        NaN
 879        NaN
 881        NaN
 880        NaN
 439        NaN
 438        NaN
 437        NaN
 436        NaN
 882        NaN
 440        NaN
 441     1500.0
 883

In [1200]:
intra.to_csv('intra_state_1')